In [1]:
from PIL import Image
import pytesseract
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [2]:
# Please copy and paste the tesseract path here:
pytesseract.pytesseract.tesseract_cmd = ''

# Example:
# pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'


In [3]:
def img_to_csv(filename):
    # Loading and pre-processing the input image
    img = Image.open(filename)
    img = img.convert('L', dither=Image.NONE)
    
    # Converting image to text data
    custom_config = '-c preserve_interword_spaces=1'    
    text = pytesseract.image_to_string(img, config=custom_config, lang='eng')

    # Initializing the output dataframe
    columns = ['Transaction_date', 'Pet_name', 'Description', 'Net_price', 'File_name']
    output = pd.DataFrame(columns=columns)
    Pet_name=''

    # Parsing through each line
    for line in text.split('\n'):
    
        entry = line.split(' ') # Splitting the line into words
        
        if re.search(r'payment', line, flags=re.IGNORECASE):
            break
        
        # Extracting the date and the price
        if re.match(r'\d+[-/]\d+', entry[0]) and re.match(r'-?\d+\.\d{2}', entry[-1]):
            Transaction_date = entry[0]
            Net_price = entry[-1]
        
            # Extracting the pet name
            if entry[1]:
                Pet_name = entry[1]
        
            # Extracting the description
            line = line.replace(Transaction_date, '')
            line = line.replace(Net_price, '')
            line = line.replace(Pet_name, '')
            line = re.sub(r'-?\d+\.\d{2}','', line)
            line = line.strip()
            if re.match('\d+', line[0]):
                line = line.split(' ', 1)[1]
            
        
            output = output.append({'Transaction_date': Transaction_date, 'Net_price': Net_price,
                                    'Pet_name': Pet_name, 'File_name': filename, 'Description':line
                                   }, ignore_index=True)
        
    
    output.to_csv('~/Desktop/output.csv', index=False) # Saving the output as .csv file on desktop
    return output

In [4]:
img_to_csv('Img1.PNG')

,Transaction_date,Pet_name,Description,Net_price,File_name
0,02-23-21,BABY,Senior Citizen Discount 10%,-21.93,Img1.PNG
1,02-23-21,BABY,NAIL TRIM WIEXAM,9.50,Img1.PNG
2,02-23-21,BABY,HEARTWORM /:TICK PANEL- ANNUAL,43.10,Img1.PNG
3,02-23-21,BABY,CANINE BORDETELLA PO ANNUAL,27.90,Img1.PNG
4,02-23-21,BABY,CANINE LYME ANNUAL,32.60,Img1.PNG
5,02-23-21,BABY,CANINE LEPTOSPIROSIS ANNUAL,27.90,Img1.PNG
6,02-23-21,BABY,CANINE DISTEMPER/PARVO 3YR (DAP3),27.90,Img1.PNG
7,02-23-21,BABY,EXAM = SENIOR PET EXAM,50.40,Img1.PNG
